# AM1054 Continuous Assessment 2

## Due Friday 30 November 2018, 9am.

In [1]:
# execute this cell before you start the assignment

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from ipywidgets.widgets import interact
sym.init_printing(use_latex="mathjax")
x, y, z, t = sym.symbols('x y z t')

ModuleNotFoundError: No module named 'sympy'

No late assignments will be accepted. Strictly individual (no team) solutions are required. 

Place your solution as file `CA2.ipynb` into the folder `AM1054\CA2` of your `Z:` drive. 

Write `std_nr='11XXXXXXX'` (where 11XXXXXXX is your student number) into the next cell:

In [ ]:
std_nr='118364841'

In [ ]:
assert(int(std_nr))

Write your solutions in the cell below each question. After completing the assignment, check that you can execute the whole assignment by selecting `Kernel -> Restart & Run All`. Make sure that all requested outputs appear as expected after you do this. 

# Question 1: Symbolic calculation

Consider the function
\begin{equation}
p_w(z,t)=\frac{1}{\sqrt{\pi \left(1-\exp\left[-2 t\right]\right)}}
\exp\left[-\frac{\left(z-\exp\left[-t\right]\right)^{2}}{1-
\exp\left[-2t\right]}\right]
\end{equation}

a) Define a python variable `p_w` which is a `sympy` expression for $p_w(z,t)$ with sympy symbols `z` and `t`.

In [ ]:
p_w = (1/sym.sqrt(sym.pi*(1-sym.exp(-2*t))))*sym.exp(-((z-sym.exp(-t))**2)/(1-sym.exp(-2*t)))

p_w

In [ ]:
np.testing.assert_allclose(float(p_w.subs({z:0, t:1}).n()), 0.518831632096486)

b) Show that $p(z,t)$ fulfills the partial differential equation 
\begin{equation}
\partial_{t}p(z,t)=\partial_{z}\left[zp(z,t|x,0)\right]+ \frac{1}{2} \partial_{z}^{2}p(z,t|x,0)
\end{equation} 

Do this by defining two Python variables `LHS` and `RHS` which contain the left hand side and right hand side of this equation. Do all differentiations using `sympy`, do not differentiate by hand! Then show that the difference LHS - RHS vanishes. 


In [ ]:
LHS = sym.diff(p_w, t)

RHS = sym.diff(z * p_w, z) + 0.5*sym.diff(sym.diff(p_w, z), z)


sym.simplify(LHS - RHS)

In [ ]:
assert(sym.simplify(LHS-RHS) == 0)

c) Plot $p(z,t)$ for the three values $t=0.1, 1, 10$ in a $p(z,t)$ versus $z$ diagram. 

In [ ]:
myx = np.linspace(0,10)

x1 = p_w.subs({t:0.1})
x2 = p_w.subs({t:1})
x3 = p_w.subs({t:10})

fx1 = sym.lambdify(z, x1)
fx2 = sym.lambdify(z, x2)
fx3 = sym.lambdify(z, x3)

myx = np.linspace(-10,10, 100)

plt.plot(myx, fx1(myx))
plt.plot(myx, fx2(myx))
plt.plot(myx, fx3(myx))

## Question 2

The spherical [Bessel functions](https://en.wikipedia.org/wiki/Bessel_function#Spherical_Bessel_functions:_jn.2C_yn) $j_n(r)$ (for $r>0$) can be defined through

\begin{equation}
j_{n-1}(r) = \left.\left[\frac{\partial^n}{\partial t^n} \frac{\cos \left(\sqrt{r^{2}-2rt}\right)}{r}\right]\right|_{t=0}
\end{equation}

a) Use this definition and sympy to calculate algebraic expressions for $j_0(r)$, $j_1(r)$, and $j_{10}(r)$ symbolically.  Assign these expressions to the Python variables `j0`, `j1` and `j10` respectively. 

Hint: Use `r = sym.symbols("r", positive=True)`

In [ ]:
r = sym.symbols("r", positive=True)
# replace the following with correct expressions:
j = sym.cos(sym.sqrt(r**2-2*r*t))/r

j0 = j.diff(t,1)

j1 = j.diff(t,2)

j10 = j.diff(t,11)

In [ ]:
assert(j0.limit(r,0)==1)
assert(j1.limit(r,0)==0)
assert(j10.limit(r,0)==0)

b) Plot $j_0(r)$, $j_1(r)$, and $j_{10}(r)$ for $r\in (0,20]$. Limit the y-axis to the interval $[-1,1]$ and add an appropriate legend. 

In [ ]:
fj0 = sym.lambdify(r, j0.subs({t:0}))
fj1 = sym.lambdify(r, j1.subs({t:0}))
fj10 = sym.lambdify(r, j10.subs({t:0}))

myx = np.linspace(0,20)

plt.plot(myx, fj0(myx))
plt.plot(myx, fj1(myx))
plt.plot(myx, fj10(myx))


plt.ylim(-1,1)

plt.legend(("j0(r)","j1(r)","j10(r)"))

c) Show that $j_{10}(r)$ fulfills the equation
\begin{equation}
r^{2}{\frac {d^{2}j_{10}(r)}{dr^{2}}}+2r{\frac {dj_{10}(r)}{dr}}+[r^{2}-10(10+1)]j_{10}(r)=0.
\end{equation}

In [ ]:
assert(sym.simplify(r**2*j10.diff(r,2) + 2*r*j10.diff(r,1) + (r**2-10*(10+1))*j10) == 0)

## Question 3: Integration

In the lecture we have introduced the Simpson method for integrating a function $f(x)$ over a given interval $[l,r]$. George Boole (the first Professor of Mathematics in Cork) suggested an alternative rule of integration. For a given $N$ (which is required to be divisible by 4) approximate the integral as follows
\begin{equation}
 \int_l^r f(x)dx \approx 
  \frac{2h}{45}\left[ 7 f_0 + 32 \left(\sum_{k=0}^{N/2-1} f_{2k+1}\right) + 12 \left(\sum_{k=0}^{N/4-1} f_{4k+2}\right)+ 14 \left(\sum_{k=1}^{N/4-1} f_{4k}\right) + 7 f_N \right]. 
\end{equation} where $h=(r-l)/N$ and $f_k = f(l+kh)$ as for the Simpson rule.

a) Write a python function `integrate_boole(f,(l,r),N)` which implements this rule. 

In [ ]:
def integrate_boole(f,lr,N):
    (l, r) = lr
    h = (r-l)/N
    xN = np.linspace(l, r, N+1)
    fN = f(xN)
    
    return (2*h/45)*(7*fN[0] + 32*sum(fN[1:-1:2]) + 12*sum(fN[2:-1:4]) + 14*sum(fN[4:-1:4]) + 7*fN[-1])

In [ ]:
# Please do not delete this cell

b) Using `integrate_boole()` calculate an approximation for the integral
\begin{equation}
\int_0^{\pi} \sin(x)dx 
\end{equation}

Use $N=8$ and the "lambda" construction. Only one line of code is allowed in the following cell. Assign this number to the Python variable `sine_int`

In [ ]:
sine_int = integrate_boole(sym.lambdify(x, sym.sin(x)), (0,np.pi), 8)
sine_int

In [ ]:
sym.integrate(sym.sin(x), (x,0,np.pi))

In [ ]:
np.testing.assert_allclose(sine_int, 2,rtol=1e-2)

c) Now consider a more interesting function given in python notation as

    def f(x):
        return x*np.sin(x)/(1+np.cos(x)**2)

Plot this function in the interval from $0$ to $\pi$. 

In [ ]:
def f(x):
    return x*np.sin(x)/(1+np.cos(x)**2)

myx = np.linspace(0, np.pi)

plt.plot(myx, f(myx))

d) It is analytically known that 
\begin{equation}
\int_0^\pi f(x)dx = \frac{\pi^2}{4}
\end{equation}

What approximate value do you obtain for this integral using `integrate_boole()` and `N=8`? What is the relative error of value?

In [ ]:
f_approx = integrate_boole(f, (0,np.pi),8)
f_exact = np.pi**2/4
relerror = abs((f_approx-f_exact)/f_exact)
print("The approx value is:", f_approx)
print("relative error is:", relerror)

e) Calculate the relative error for the values $N=8, 20, 40, 100, 200, 400, 1000, 2000, 4000$. Present your result in the form of a graph, where the relative error is plotted against $N$ using red dots. Use log scale for both axes and label them.

In [ ]:
N_values = np.array([8,20,40,100,200,400,1000,2000,4000])

for N in N_values:
    f_approx_graph = integrate_boole(f, (0,np.pi), N)
    relerror_graph = abs((f_approx_graph-f_exact)/f_exact)
    
    plt.loglog(N,relerror_graph, 'ro')
    
    
plt.xlabel("N value")
plt.ylabel("relative error")

f) What is the order of convergence of `integrate_boole()`? Find out be adding appropriate plots of $1/N^d$ to the last graph.

In [ ]:
N_values = np.array([8,20,40,100,200,400,1000,2000,4000])

for N in N_values:
    f_approx_graph = integrate_boole(f, (0,np.pi), N)
    relerror_graph = abs((f_approx_graph-f_exact)/f_exact)
    
    plt.loglog(N,relerror_graph, 'ro')

    
    
plt.xlabel("N value")
plt.ylabel("relative error")

plt.plot(N_values, N_values**(-2.0))
plt.plot(N_values, N_values**(-3.0))
plt.plot(N_values, N_values**(-4.0))
plt.plot(N_values, N_values**(-5.0))
plt.plot(N_values, N_values**(-6.0))

print("The order of convergence is 6")

g) In practice it is a nuisance that we have to specify `N` when we want to integrate a given function. Instead one would prefer to have a function `integrate_boole_eps(f,lr,eps)`, which takes as one of it's argument `eps` the desired relative error.  The algorithm itself should then be able to decide, how many intervals `N` are required to obtain a required accuracy `eps`. 

Implement a function `integrate_boole_eps(f,lr,eps)` which uses the following strategy to calculate the integral with precision `eps`

1. Calculate `integrate_boole(f,lr,N)` for `N` equals 8 and 16. If the relative difference between these two results is less than `eps` return the result for `N=16`. 
2. Else calculate `integrate_boole(f,lr,N)` for `N=32`. If the relative difference between this result and the one for `N=16` is less than `eps` return the result for `N=32`
3. Go on successively doubling the number `N` and calculating `integrate_boole(f,(l,r),N)`. Whenever you find that the relative difference compared to the previous result has dropped below `eps`, return your most accurate result. 
4. Do not iterate beyond $N_{\max}=2^{12}$. If you reach this many iterations without finding an accurate result, print out a warning and return the most accurate result obtained. 


In [ ]:
def integrate_boole_eps(f, lr, eps):
    n = 4
    n_max = 12
    
    approx1 = integrate_boole(f, lr, 2**(n-1))
    approx2 = integrate_boole(f, lr, 2**n)
    
    reldiff = abs(approx1-approx2)
    min_reldiff = reldiff
    n_min = n
    
    while(reldiff > eps and n<n_max):
        n+=1
        
        approx1 = integrate_boole(f, lr, 2**(n-1))
        approx2 = integrate_boole(f, lr, 2**n)
        
        reldiff = abs(approx1 - approx2)
        
        if reldiff < min_reldiff:
            min_reldiff = reldiff
            n_min = n
        
    if min_reldiff > eps:
        return("Reached maximum of iterations! The smallest relative difference is: "+ str(min_reldiff)+" for N="+str(n_min))
        
    
    return ("The relative difference is: "+str(reldiff)+" for N="+str(n))

        

In [ ]:
# Please do not delete this cell

h) Test the function `integrate_boole_eps()`  by integrating `integrate_boole_eps(f,(0,np.pi),eps)` for `eps = 1e-5`, `1e-8`, `1e-12` and `1e-18`.  What are the actual relative errors in these cases?

In [ ]:
integrate_boole_eps(f,(0,np.pi),1e-16)